In [ ]:
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

In [ ]:
df = pd.read_csv("/content/BankChurners.csv")
df.shape

(10127, 23)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 23 columns):
 #   Column                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                              --------------  -----  
 0   CLIENTNUM                                                                                                                           10127 non-null  int64  
 1   Attrition_Flag                                                                                                                      10127 non-null  object 
 2   Customer_Age                                                                                                                        10127 non-null  int64  
 3   Gender                                                                           

In [ ]:
df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
         'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
         'CLIENTNUM'],axis=1,inplace=True)

In [ ]:
df.shape

(10127, 20)

In [ ]:
df['Attrition_Flag'].unique()

array(['Existing Customer', 'Attrited Customer'], dtype=object)

In [ ]:
label_map = {'Attrition_Flag':{'Existing Customer':0,
                               'Attrited Customer':1}}
df.replace(label_map,inplace=True)

/tmp/ipython-input-7-3213557999.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(label_map,inplace=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  int64  
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

In [ ]:
df_ohe = pd.get_dummies(df)

In [ ]:
df_ohe.shape

(10127, 38)

In [ ]:
#Split X & Y
X = df_ohe.drop(['Attrition_Flag'], axis=1)
Y = df_ohe['Attrition_Flag']

In [ ]:
X.shape

(10127, 37)

In [ ]:
Y.shape

(10127,)

In [ ]:
Y.value_counts(normalize=True)

,proportion
Attrition_Flag,
0,0.83934
1,0.16066


In [ ]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.3,
                                                    random_state=7,
                                                    stratify=Y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(7088, 37) (3039, 37) (7088,) (3039,)


In [ ]:
n_features = X_train.shape[1]

In [ ]:
# define model
model1 = Sequential()
model1.add(Input((n_features,)))
model1.add(Dense(5, activation='relu'))
model1.add(Dense(4, activation='tanh'))
model1.add(Dense(20, activation='linear'))
model1.add(Dense(14, activation='gelu'))
model1.add(Dense(25, activation='sigmoid'))
model1.add(Dense(1, activation='sigmoid'))

In [ ]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 5)              │           190 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14)             │           294 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 25)             │           375 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,009 (3.94 KB)

 Trainable params: 1,009 (3.94 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy',
               metrics=['accuracy','precision','recall','f1_score'])

In [ ]:
# fit the model (traning)
history = model1.fit( X_train, y_train, epochs=10, batch_size=32,
                     validation_split=0.3, verbose=2,shuffle=True)

Epoch 1/10
156/156 - 8s - 48ms/step - accuracy: 0.7811 - f1_score: 0.2720 - loss: 0.5008 - precision: 0.1378 - recall: 0.0743 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.4435 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
156/156 - 1s - 6ms/step - accuracy: 0.8426 - f1_score: 0.2720 - loss: 0.4202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.4379 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
156/156 - 1s - 7ms/step - accuracy: 0.8426 - f1_score: 0.2720 - loss: 0.4193 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.4361 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
156/156 - 1s - 9ms/step - accuracy: 0.8426 - f1_score: 0.2720 - loss: 0.4184 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.4361 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epo

# Model 2


In [ ]:
# define model
model1 = Sequential()
model1.add(Input((n_features,)))
model1.add(Dense(5, activation='relu',
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer='l2', bias_regularizer=None,
                 ))
model1.add(Dense(1, activation='sigmoid',
                 kernel_initializer='glorot_normal',
                 bias_initializer='ones',
                 kernel_regularizer='l1', bias_regularizer=None,))

In [ ]:
model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 5)              │           190 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 196 (784.00 B)

 Trainable params: 196 (784.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compile the model
# optimizer :: sgd , rmsprop, adam   #momentum
model1.compile(optimizer='rmsprop', loss='binary_crossentropy',
               metrics=['accuracy','precision','recall','f1_score'])

In [ ]:
# fit the model (traning)
history = model1.fit( X_train, y_train, epochs=200, batch_size=1500,
                     validation_split=0.3, verbose=2,shuffle=False)

Epoch 1/200
4/4 - 3s - 734ms/step - accuracy: 0.5539 - f1_score: 0.1276 - loss: 278.6494 - precision: 0.0918 - recall: 0.2061 - val_accuracy: 0.5468 - val_f1_score: 0.1274 - val_loss: 257.5983 - val_precision: 0.0916 - val_recall: 0.1899
Epoch 2/200
4/4 - 0s - 122ms/step - accuracy: 0.5731 - f1_score: 0.1340 - loss: 229.8210 - precision: 0.0985 - recall: 0.2100 - val_accuracy: 0.5637 - val_f1_score: 0.1327 - val_loss: 221.2414 - val_precision: 0.0997 - val_recall: 0.1983
Epoch 3/200
4/4 - 1s - 130ms/step - accuracy: 0.5854 - f1_score: 0.1412 - loss: 198.7226 - precision: 0.1037 - recall: 0.2138 - val_accuracy: 0.5769 - val_f1_score: 0.1404 - val_loss: 191.5535 - val_precision: 0.1072 - val_recall: 0.2067
Epoch 4/200
4/4 - 0s - 76ms/step - accuracy: 0.5942 - f1_score: 0.1479 - loss: 173.7263 - precision: 0.1096 - recall: 0.2215 - val_accuracy: 0.5886 - val_f1_score: 0.1554 - val_loss: 166.9166 - val_precision: 0.1193 - val_recall: 0.2263
Epoch 5/200
4/4 - 0s - 64ms/step - accuracy: 0.60

# Task

1. Create a NN with 3 hidden layers, all relu activation, rmsprop optimizer, epochs 100, batch size=32, regularization 'l2' for all hidden layers


# Model 3

- Change learning rate

- momentum optimizer

In [ ]:
from keras.optimizers import SGD, Adam

In [ ]:
# Momemntum optimizer
sgd1 = SGD(learning_rate=0.0001,momentum=0.99 )

In [ ]:
a1 = Adam(learning_rate=0.0001 )

In [ ]:
# define model
model1 = Sequential()
model1.add(Input((n_features,)))
model1.add(Dense(5, activation='relu',))
model1.add(Dense(1, activation='sigmoid',))

In [ ]:
model1.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 5)              │           190 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 196 (784.00 B)

 Trainable params: 196 (784.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model1.compile(optimizer=sgd1, loss='binary_crossentropy',
               metrics=['accuracy','precision','recall','f1_score'])

In [ ]:
# fit the model (traning)
history = model1.fit( X_train, y_train, epochs=100, batch_size=32,
                     validation_split=0.3, verbose=2,shuffle=False)

Epoch 1/100
156/156 - 3s - 20ms/step - accuracy: 0.8321 - f1_score: 0.2692 - loss: 79.0751 - precision: 0.0938 - recall: 0.0077 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.6114 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
156/156 - 1s - 6ms/step - accuracy: 0.8426 - f1_score: 0.2720 - loss: 0.5630 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.5296 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
156/156 - 1s - 9ms/step - accuracy: 0.8426 - f1_score: 0.2720 - loss: 0.4972 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.4870 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100
156/156 - 1s - 5ms/step - accuracy: 0.8426 - f1_score: 0.2720 - loss: 0.4650 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8317 - val_f1_score: 0.2881 - val_loss: 0.4681 - val_precision: 0.0000e+00 - val_recall: 0.0000e+0

#Model 4

- Early stopping

In [ ]:
# define model
model1 = Sequential()
model1.add(Input((n_features,)))
model1.add(Dense(5, activation='relu',))
model1.add(Dense(1, activation='sigmoid',))

In [ ]:
model1.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 5)              │           190 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 196 (784.00 B)

 Trainable params: 196 (784.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model1.compile(optimizer='adam', loss='binary_crossentropy',
               metrics=['accuracy','precision','recall','f1_score'])

In [ ]:
# Here it stops the training if for 5 continuous epochs val_loss increases
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=10)

In [ ]:
# fit the model (traning)
history = model1.fit( X_train, y_train, epochs=100, batch_size=32,
                     validation_split=0.3, verbose=2,
                      shuffle=False, callbacks=[es])

Epoch 1/100
156/156 - 4s - 23ms/step - accuracy: 0.6829 - f1_score: 0.1845 - loss: 60.8984 - precision: 0.1360 - recall: 0.1895 - val_accuracy: 0.7767 - val_f1_score: 0.1573 - val_loss: 17.7285 - val_precision: 0.1020 - val_recall: 0.0419
Epoch 2/100
156/156 - 1s - 6ms/step - accuracy: 0.7992 - f1_score: 0.1664 - loss: 12.2433 - precision: 0.1331 - recall: 0.0499 - val_accuracy: 0.8190 - val_f1_score: 0.1684 - val_loss: 6.5275 - val_precision: 0.1538 - val_recall: 0.0168
Epoch 3/100
156/156 - 2s - 11ms/step - accuracy: 0.8148 - f1_score: 0.2649 - loss: 2.1052 - precision: 0.2346 - recall: 0.0781 - val_accuracy: 0.8402 - val_f1_score: 0.2881 - val_loss: 0.7306 - val_precision: 0.7045 - val_recall: 0.0866
Epoch 4/100
156/156 - 2s - 12ms/step - accuracy: 0.8420 - f1_score: 0.2721 - loss: 0.6322 - precision: 0.4962 - recall: 0.2510 - val_accuracy: 0.8472 - val_f1_score: 0.2881 - val_loss: 0.6498 - val_precision: 0.7324 - val_recall: 0.1453
Epoch 5/100
156/156 - 1s - 5ms/step - accuracy: 0.

# Task

- Make your model with various parameters!!!

# Output layer with Softmax

In [ ]:
df = pd.read_csv("/content/BankChurners.csv")
df.shape

(10127, 23)

In [ ]:
df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
         'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
         'CLIENTNUM'],axis=1,inplace=True)

In [ ]:
df_ohe = pd.get_dummies(df)

In [ ]:
df_ohe.shape

(10127, 39)

In [ ]:
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Customer_Age                      10127 non-null  int64  
 1   Dependent_count                   10127 non-null  int64  
 2   Months_on_book                    10127 non-null  int64  
 3   Total_Relationship_Count          10127 non-null  int64  
 4   Months_Inactive_12_mon            10127 non-null  int64  
 5   Contacts_Count_12_mon             10127 non-null  int64  
 6   Credit_Limit                      10127 non-null  float64
 7   Total_Revolving_Bal               10127 non-null  int64  
 8   Avg_Open_To_Buy                   10127 non-null  float64
 9   Total_Amt_Chng_Q4_Q1              10127 non-null  float64
 10  Total_Trans_Amt                   10127 non-null  int64  
 11  Total_Trans_Ct                    10127 non-null  int64  
 12  Tota

In [ ]:
X = df_ohe.drop(['Attrition_Flag_Attrited Customer',
                 'Attrition_Flag_Existing Customer'], axis=1)
Y = df_ohe[['Attrition_Flag_Attrited Customer',
                 'Attrition_Flag_Existing Customer']]

In [ ]:
X.shape,Y.shape

((10127, 37), (10127, 2))

In [ ]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.3,
                                                    random_state=7,
                                                    stratify=Y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(7088, 37) (3039, 37) (7088, 2) (3039, 2)


In [ ]:
n_features = X_train.shape[1]

In [ ]:
# define model
model1 = Sequential()
model1.add(Input((n_features,)))
model1.add(Dense(5, activation='relu'))
model1.add(Dense(2, activation='softmax'))

In [ ]:
model1.compile(optimizer='adam', loss='binary_crossentropy',
               metrics=['accuracy','precision','recall','f1_score'])

In [ ]:
# fit the model (traning)
history = model1.fit( X_train, y_train, epochs=5, batch_size=32,
                     validation_split=0.3, verbose=2,
                      shuffle=True)

Epoch 1/5
156/156 - 8s - 52ms/step - accuracy: 0.8422 - f1_score: 0.4572 - loss: 0.4677 - precision: 0.8422 - recall: 0.8422 - val_accuracy: 0.8317 - val_f1_score: 0.4541 - val_loss: 0.4499 - val_precision: 0.8317 - val_recall: 0.8317
Epoch 2/5
156/156 - 2s - 11ms/step - accuracy: 0.8426 - f1_score: 0.4573 - loss: 0.4324 - precision: 0.8426 - recall: 0.8426 - val_accuracy: 0.8317 - val_f1_score: 0.4541 - val_loss: 0.4507 - val_precision: 0.8317 - val_recall: 0.8317
Epoch 3/5
156/156 - 1s - 8ms/step - accuracy: 0.8426 - f1_score: 0.4573 - loss: 0.4331 - precision: 0.8426 - recall: 0.8426 - val_accuracy: 0.8317 - val_f1_score: 0.4541 - val_loss: 0.4500 - val_precision: 0.8317 - val_recall: 0.8317
Epoch 4/5
156/156 - 1s - 8ms/step - accuracy: 0.8426 - f1_score: 0.4573 - loss: 0.4339 - precision: 0.8426 - recall: 0.8426 - val_accuracy: 0.8317 - val_f1_score: 0.4541 - val_loss: 0.4527 - val_precision: 0.8317 - val_recall: 0.8317
Epoch 5/5
156/156 - 1s - 6ms/step - accuracy: 0.8426 - f1_scor